In [23]:
import re


def process_query(query):
    tables_needed = set()
    plate_number = None

    # Define patterns to identify the type of query
    patterns = {
        "current_location": [
            r"where is vehicle ([A-Za-z0-9 ]+)\??",
            r"what is the current location of vehicle ([A-Za-z0-9 ]+)\??",
            r"location of vehicle ([A-Za-z0-9 ]+)\??",
            r"current location of vehicle ([A-Za-z0-9 ]+)\??",
            r"vehicle ([A-Za-z0-9 ]+) location\??",
        ],
        "dispatch_time": [
            r"what time was vehicle ([A-Za-z0-9 ]+) dispatched\??",
            r"when was vehicle ([A-Za-z0-9 ]+) dispatched\??",
            r"dispatch time of vehicle ([A-Za-z0-9 ]+)\??",
            r"vehicle ([A-Za-z0-9 ]+) dispatch time\??",
        ],
        "arrival_time": [
            r"what time did it arrive in the customer site\??",
            r"when did vehicle ([A-Za-z0-9 ]+) arrive at the customer site\??",
            r"arrival time of vehicle ([A-Za-z0-9 ]+) at customer site\??",
            r"vehicle ([A-Za-z0-9 ]+) customer site arrival time\??",
        ],
    }

    for query_type, pattern_list in patterns.items():
        for pattern in pattern_list:
            match = re.search(pattern, query.lower())
            if match and len(match.groups()) > 0 and match.group(1):
                plate_number = match.group(1).strip()
                if query_type == "current_location":
                    tables_needed.update(["vehicles", "movements"])
                elif query_type == "dispatch_time":
                    tables_needed.update(["vehicles", "movements"])
                elif query_type == "arrival_time":
                    tables_needed.update(["vehicles", "locations", "movements"])
                return {
                    "plate_number": plate_number,
                    "query_type": query_type,
                    "tables_needed": list(tables_needed),
                }

    return {"plate_number": None, "query_type": None, "tables_needed": []}

In [24]:
import os

from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize the language model
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", api_key=OPENAI_API_KEY)

# Define the prompt template
prompt_template = """
You are an intelligent assistant for a logistics company. Categorize the query and extract relevant information such as the plate number.
Query: {query}

Response:
Query Type: (current_location, dispatch_time, arrival_time, unknown)
Plate Number: (if available)
"""

prompt = PromptTemplate(input_variables=["query"], template=prompt_template)

# Define the chain
chain = LLMChain(prompt=prompt, llm=llm)


# Define the function to process the query using LangChain
def process_query_langchain(query):
    response = chain.run({"query": query})

    # Extract relevant details from the response
    query_type = None
    plate_number = None
    tables_needed = set()

    # Parse the response
    if "Query Type:" in response:
        query_type_line = response.split("Query Type:")[1].split("\n")[0].strip()
        query_type = query_type_line.replace("(", "").replace(")", "").strip()

    if "Plate Number:" in response:
        plate_number_line = response.split("Plate Number:")[1].split("\n")[0].strip()
        if "if available" not in plate_number_line:
            plate_number = plate_number_line.replace("(", "").replace(")", "").strip()

    if query_type == "current_location":
        tables_needed.update(["vehicles", "movements"])
    elif query_type == "dispatch_time":
        tables_needed.update(["vehicles", "movements"])
    elif query_type == "arrival_time":
        tables_needed.update(["vehicles", "locations", "movements"])

    return {
        "plate_number": plate_number,
        "query_type": query_type,
        "tables_needed": list(tables_needed),
    }

In [25]:
# Example queries
queries = [
    "Where is vehicle ABC 1234?",
    "What is the current location of vehicle ABC 1234?",
    "Location of vehicle ABC 1234?",
    "What time was vehicle ABC 1234 dispatched?",
    "When was vehicle ABC 1234 dispatched?",
    "What time did it arrive in the customer site?",
    "When did vehicle ABC 1234 arrive at the customer site?",
    "What is the current location of vehicle?",
]

for query in queries:
    print(process_query(query))
    print(process_query_langchain(query))

{'plate_number': 'abc 1234', 'query_type': 'current_location', 'tables_needed': ['movements', 'vehicles']}
{'plate_number': None, 'query_type': None, 'tables_needed': []}
{'plate_number': 'abc 1234', 'query_type': 'current_location', 'tables_needed': ['movements', 'vehicles']}
{'plate_number': None, 'query_type': None, 'tables_needed': []}
{'plate_number': 'abc 1234', 'query_type': 'current_location', 'tables_needed': ['movements', 'vehicles']}
{'plate_number': 'ABC 1234', 'query_type': None, 'tables_needed': []}
{'plate_number': 'abc 1234', 'query_type': 'dispatch_time', 'tables_needed': ['movements', 'vehicles']}
{'plate_number': None, 'query_type': None, 'tables_needed': []}
{'plate_number': 'abc 1234', 'query_type': 'dispatch_time', 'tables_needed': ['movements', 'vehicles']}
{'plate_number': 'ABC 1234', 'query_type': None, 'tables_needed': []}
{'plate_number': None, 'query_type': None, 'tables_needed': []}
{'plate_number': None, 'query_type': None, 'tables_needed': []}
{'plate_num